In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
    .appName("YelpAZBusinessAnalysis")\
    .getOrCreate()

24/11/19 12:50:14 WARN Utils: Your hostname, yelp resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s17)
24/11/19 12:50:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/19 12:50:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:


yelp_df = spark.read.json("file:///home/dps/Documents/yelp_dataset/yelp_academic_dataset_business.json")

yelp_df.printSchema()
yelp_df.show(5)

arizona_df = yelp_df.filter(yelp_df["state"] == "AZ")
arizona_df.show(10)

arizona_df.write.parquet("file:///home/dps/Documents/temp/arizona_businesses.parquet")

yelp_df.select("categories").distinct().show(truncate=False)

breweries_df = arizona_df.filter(arizona_df["categories"].contains("Breweries"))
breweries_df.show(5)

breweries_df.createOrReplaceTempView("breweries")


root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

+-----------------------------------------------------------------------------------------------------------------------------+
|categories                                                                                                                   |
+-----------------------------------------------------------------------------------------------------------------------------+
|Arts & Entertainment, Event Planning & Services, Venues & Event Spaces, Museums                                              |
|Food, Convenience Stores, Restaurants, Food Delivery Services, Local Flavor, Delis                                           |
|Automotive, Auto Repair, Towing, Body Shops, Oil Change Stations, Auto Parts & Supplies, Tires                               |
|Makeup Artists, Hair Salons, Beauty & Spas                                                                                   |
|Breweries, American (New), Restaurants, Burgers, American (Traditional), Gastropubs, Brewpubs, Food, Fi

In [5]:
spark.sql("""
SELECT name, stars, review_count
FROM breweries
ORDER BY stars DESC, review_count DESC
LIMIT 10
""").show()

+--------------------+-----+------------+
|                name|stars|review_count|
+--------------------+-----+------------+
|     Serial Grillers|  4.5|         986|
|  Dragoon Brewing Co|  4.5|         182|
| Pueblo Vida Brewing|  4.5|         172|
|  Arizona Beer House|  4.5|         164|
|Crooked Tooth Bre...|  4.5|         163|
|1912 Brewing Company|  4.5|          87|
|   Button Brew House|  4.5|          62|
|Dillinger Brewing...|  4.5|          61|
| Iron John's Brewing|  4.5|          50|
|Ten Fifty-Five Br...|  4.5|          48|
+--------------------+-----+------------+



In [ ]:
spark.sql("""
SELECT postal_code, COUNT(*) AS num_breweries
FROM breweries
GROUP BY postal_code
ORDER BY num_breweries DESC
LIMIT 10
""").show()

+-----------+-------------+
|postal_code|num_breweries|
+-----------+-------------+
|      85705|            8|
|      85719|            6|
|      85701|            5|
|      85713|            4|
|      85712|            3|
|      85741|            3|
|      85711|            2|
|      85749|            2|
|      85710|            2|
|      85756|            2|
+-----------+-------------+



In [8]:
spark.sql("""
SELECT
    name,
    stars,
    review_count,
    ROUND((stars * 0.7 + LOG10(review_count + 1) * 2), 2) AS breweries_score
FROM breweries
WHERE categories LIKE '%Breweries%'
ORDER BY breweries_score DESC
LIMIT 10
""").show()

+--------------------+-----+------------+---------------+
|                name|stars|review_count|breweries_score|
+--------------------+-----+------------+---------------+
|     Serial Grillers|  4.5|         986|           9.14|
|      Barrio Brewing|  4.0|         611|           8.37|
|                1702|  4.0|         337|           7.86|
|Sentinel Peak Bre...|  4.0|         304|           7.77|
|  Dragoon Brewing Co|  4.5|         182|           7.67|
| Pueblo Vida Brewing|  4.5|         172|           7.63|
|Crooked Tooth Bre...|  4.5|         163|           7.58|
|  Arizona Beer House|  4.5|         164|           7.58|
|Gentle Ben’s Brewery|  3.5|         359|           7.56|
| Zpizza - Oro Valley|  4.0|         236|           7.55|
+--------------------+-----+------------+---------------+



In [ ]:
spark.sql("""
SELECT
    postal_code,
    COUNT(*) AS num_breweries,
    ROUND(AVG(stars), 2) AS avg_rating
FROM breweries
WHERE attributes.BusinessAcceptsCreditCards ='True' AND attributes.GoodForKids = 'True'
GROUP BY postal_code
ORDER BY avg_rating DESC
LIMIT 10
""").show()

+-----------+-------------+----------+
|postal_code|num_breweries|avg_rating|
+-----------+-------------+----------+
|      85714|            1|       4.5|
|      85745|            1|       4.5|
|      85712|            2|      4.25|
|      85737|            1|       4.0|
|      85704|            1|       3.5|
|      85741|            2|       3.5|
|      85701|            1|       3.5|
|      85719|            3|       3.5|
|      85711|            2|      3.25|
|      85705|            2|      3.25|
+-----------+-------------+----------+



In [16]:
spark.sql("""
SELECT
    name,
    postal_code,
    stars,
    review_count,
    ROUND((stars * 0.5 + 
          CASE WHEN attributes.BusinessAcceptsCreditCards = 'True' THEN 0.2 ELSE 0 END +
          CASE WHEN attributes.GoodForKids = 'True' THEN 0.1 ELSE 0 END), 2) AS weighted_score
FROM breweries
WHERE attributes.GoodForKids = 'True' AND attributes.BusinessAcceptsCreditCards = 'True'
ORDER BY weighted_score DESC, stars DESC
LIMIT 10
""").show()

+--------------------+-----------+-----+------------+--------------+
|                name|postal_code|stars|review_count|weighted_score|
+--------------------+-----------+-----+------------+--------------+
|1912 Brewing Company|      85745|  4.5|          87|          2.55|
|Harbottle Brewing...|      85714|  4.5|          34|          2.55|
|     Serial Grillers|      85712|  4.5|         986|          2.55|
|Sentinel Peak Bre...|      85712|  4.0|         304|           2.3|
|Catalina Brewing ...|      85741|  4.0|          67|           2.3|
| Zpizza - Oro Valley|      85737|  4.0|         236|           2.3|
|                1702|      85719|  4.0|         337|           2.3|
|   Firetruck Brewing|      85704|  3.5|          51|          2.05|
|Thunder Canyon Br...|      85701|  3.5|         333|          2.05|
|      TapHouse Grill|      85711|  3.5|          13|          2.05|
+--------------------+-----------+-----+------------+--------------+

